<a href="https://colab.research.google.com/github/Ofiregev/Final_Project-FetalCns/blob/main/Fetal_Cns_Final_reg_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os
import pandas as pd
import torch.nn.functional as F
from sklearn.model_selection import KFold
import numpy as np

# Mount Google Drive
drive.mount('/content/drive')

print("Part 1 has done")

Mounted at /content/drive
Part 1 has done


In [4]:
# Part 2: Define CustomDataset Class and Read CSV File
class CustomDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.data_frame.iloc[idx, 0])
        image = Image.open(img_name)
        if self.transform:
            image = self.transform(image)

        label = self.data_frame.iloc[idx, 2]  # Assuming the third column is the label

        return image, label

In [29]:

# Determine the available hardware (CPU or GPU) and set the PyTorch device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cpu


In [6]:
# Read the list of images from the CSV file
train_csv = "/content/drive/MyDrive/FinalProject/training_set_pixel_size_and_HC.csv"
train_csv_df = pd.read_csv(train_csv)


In [26]:
train_csv_df.head()
len(train_csv_df)

999

In [8]:
# Part 3: Define Transforms and Create Dataset

transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
])
train_dataset = CustomDataset(csv_file="/content/drive/MyDrive/FinalProject/training_set_pixel_size_and_HC.csv",
                              root_dir="/content/drive/MyDrive/FinalProject/Dataset/training_set/training_set/",
                              transform=transform)

In [89]:
# Part 4: Create DataLoader
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [90]:
# Part 5: Define Model
# class Model(nn.Module):
#     def __init__(self, input_channels=1, input_size=64):
#         super(Model, self).__init__()
#         self.conv1 = nn.Conv2d(input_channels, 64, 3, padding=1)
#         self.conv2 = nn.Conv2d(64, 128, 3, padding=1)
#         self.conv3 = nn.Conv2d(128, 256, 3, padding=1)
#         self.pool = nn.MaxPool2d(2, 2)
#         conv_output_size = input_size // 2 // 2 // 2  # Adjusted for additional pooling layer
#         self.fc1 = nn.Linear(256 * conv_output_size * conv_output_size, 512)
#         self.fc2 = nn.Linear(512, 256)
#         self.fc3 = nn.Linear(256, 1)
#         self.dropout = nn.Dropout(0.3)  # Dropout probability of 0.5

#     def forward(self, x):
#         x = self.pool(F.relu(self.conv1(x)))
#         x = self.pool(F.relu(self.conv2(x)))
#         x = self.pool(F.relu(self.conv3(x)))
#         x = torch.flatten(x, 1)
#         x = F.relu(self.fc1(x))
#         x = self.dropout(x)
#         x = F.relu(self.fc2(x))
#         x = self.dropout(x)
#         x = self.fc3(x)
#         return x
class Model(nn.Module):
    def __init__(self, input_channels=1, input_size=64):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(input_channels, 64, 3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv3 = nn.Conv2d(128, 256, 3, padding=1)
        self.conv4 = nn.Conv2d(256, 512, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        conv_output_size = input_size // 2 // 2 // 2 // 2  # Adjusted for additional pooling layers
        self.fc1 = nn.Linear(512 * conv_output_size * conv_output_size, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 1)
        self.dropout = nn.Dropout(0.3)
        self.batchnorm1 = nn.BatchNorm2d(64)
        self.batchnorm2 = nn.BatchNorm2d(128)
        self.batchnorm3 = nn.BatchNorm2d(256)
        self.batchnorm4 = nn.BatchNorm2d(512)

    def forward(self, x):
        x = self.pool(F.relu(self.batchnorm1(self.conv1(x))))
        x = self.pool(F.relu(self.batchnorm2(self.conv2(x))))
        x = self.pool(F.relu(self.batchnorm3(self.conv3(x))))
        x = self.pool(F.relu(self.batchnorm4(self.conv4(x))))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        x = self.dropout(x)
        x = self.fc4(x)
        return x

In [91]:
model = Model(input_channels=1, input_size=64)
model.to(device)

Model(
  (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=8192, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=1, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (batchnorm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm4)

In [92]:
# Part 6: Define Training Parameters and Optimizer
learning_rate = 0.001
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)


In [93]:
def train(model, train_loader, optimizer, criterion, epochs, device):
    model.to(device)
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0

        for inputs, labels in train_loader:
            inputs = inputs.float().to(device)
            labels = labels.float().unsqueeze(1).to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        epoch_loss = running_loss / len(train_loader)
        print(f"Epoch {epoch + 1}, Loss: {epoch_loss:.4f}")

In [94]:
def evaluate(model, val_loader, criterion, device):
    model.eval()
    val_losses = []

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.float().to(device)
            labels = labels.float().unsqueeze(1).to(device)

            outputs = model(inputs)
            val_loss = criterion(outputs, labels)
            val_losses.append(val_loss.item())

    fold_avg_val_loss = np.mean(val_losses)
    return fold_avg_val_loss

In [95]:
k = 3
kf = KFold(n_splits=k, shuffle=True, random_state=42)
evaluation_metrics = []
epochs = 4

for fold_idx, (train_idx, val_idx) in enumerate(kf.split(train_dataset)):
    print(f"Fold {fold_idx + 1}")
    print("=========")

    train_data_fold = [train_dataset[i] for i in train_idx]
    val_data_fold = [train_dataset[i] for i in val_idx]

    train_loader_fold = DataLoader(train_data_fold, batch_size=batch_size, shuffle=True, num_workers=2)
    val_loader_fold = DataLoader(val_data_fold, batch_size=batch_size, num_workers=4)

    train(model, train_loader_fold, optimizer, criterion, epochs, device)
    fold_avg_val_loss = evaluate(model, val_loader_fold, criterion, device)
    evaluation_metrics.append(fold_avg_val_loss)

    print(f"Fold {fold_idx + 1} Validation Loss: {fold_avg_val_loss:.4f}")
    print("=========")

average_metric = np.mean(evaluation_metrics)
print("Average Evaluation Metric:", average_metric)

Fold 1
Epoch 1, Loss: 5823.8681
Epoch 2, Loss: 2347.5808
Epoch 3, Loss: 2969.2506
Epoch 4, Loss: 1925.1443
Fold 1 Validation Loss: 2292.3991
Fold 2
Epoch 1, Loss: 2384.0371
Epoch 2, Loss: 1795.5097
Epoch 3, Loss: 1725.9423
Epoch 4, Loss: 2241.7564
Fold 2 Validation Loss: 1171.4383
Fold 3
Epoch 1, Loss: 2405.6004
Epoch 2, Loss: 1648.8445
Epoch 3, Loss: 1424.1937
Epoch 4, Loss: 1902.3650
Fold 3 Validation Loss: 1806.0514
Average Evaluation Metric: 1756.629595559741


In [96]:
# Part 8: Save the trained model
model_path = "/content/drive/MyDrive/FinalProject/trained_model.pth"
torch.save(model.state_dict(), model_path)
print("Model saved successfully!")

Model saved successfully!


In [97]:
# # Load the trained model
model_path = "/content/drive/MyDrive/FinalProject/trained_model.pth"
model = Model(input_channels=1, input_size=64)
model.load_state_dict(torch.load(model_path))
model.eval()

Model(
  (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=8192, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=1, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (batchnorm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm4)

In [98]:
# Function to predict head circumference from a new image
def predict_head_circumference(image_path, model):
    # Open and preprocess the new image
    transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor()
])
    image = Image.open(image_path)
    image_tensor = transform(image).unsqueeze(0)  # Add batch dimension

    # Make prediction
    with torch.no_grad():
        output = model(image_tensor.float())

    return output.item()  # Return the predicted head circumference as a scalar

In [99]:
# Choose 10 random indices from the dataset
num_images_to_test = 500
random_indices = np.random.choice(len(train_csv_df), size=num_images_to_test, replace=False)

# Initialize a list to store the absolute errors
absolute_errors = []

# Loop through the selected images
for idx in random_indices:
    image_path = os.path.join("/content/drive/MyDrive/FinalProject/Dataset/training_set/training_set/", train_csv_df.iloc[idx, 0])
    ground_truth_circumference = train_csv_df.iloc[idx, 2]  # Assuming the third column contains the head circumference labels

    # Call the function to predict head circumference from the image
    predicted_circumference = predict_head_circumference(image_path, model)

    # Calculate the absolute error and add it to the list
    absolute_error = abs(predicted_circumference - ground_truth_circumference)
    absolute_errors.append(absolute_error)

    # Print the predicted and ground truth head circumferences
    print("Image:", image_path)
    print("Predicted Head Circumference:", predicted_circumference)
    print("Ground Truth Head Circumference:", ground_truth_circumference)
    print("Absolute Error:", absolute_error)
    print("==============================================")

# Calculate the average absolute error
average_absolute_error = sum(absolute_errors) / len(absolute_errors)
print("Average Absolute Error:", average_absolute_error)


Image: /content/drive/MyDrive/FinalProject/Dataset/training_set/training_set/786_HC.png
Predicted Head Circumference: 321.1173400878906
Ground Truth Head Circumference: 299.97
Absolute Error: 21.147340087890598
Image: /content/drive/MyDrive/FinalProject/Dataset/training_set/training_set/486_HC.png
Predicted Head Circumference: 197.7412872314453
Ground Truth Head Circumference: 192.29
Absolute Error: 5.4512872314453205
Image: /content/drive/MyDrive/FinalProject/Dataset/training_set/training_set/665_HC.png
Predicted Head Circumference: 236.04541015625
Ground Truth Head Circumference: 226.9
Absolute Error: 9.145410156249994
Image: /content/drive/MyDrive/FinalProject/Dataset/training_set/training_set/304_HC.png
Predicted Head Circumference: 237.2294158935547
Ground Truth Head Circumference: 173.92
Absolute Error: 63.3094158935547
Image: /content/drive/MyDrive/FinalProject/Dataset/training_set/training_set/024_HC.png
Predicted Head Circumference: 131.55014038085938
Ground Truth Head Circumf

In [100]:
# # Path for the new image
# new_image_path = "/content/drive/MyDrive/FinalProject/Dataset/training_set/training_set/001_HC.png"
# # Call the function to predict head circumference from the new image
# predicted_circumference = predict_head_circumference(new_image_path, model)
# print("Predicted Head Circumference:", predicted_circumference)
